# 1. Install and Import Dependencies


In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [4]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re
import torch
import requests
from bs4 import BeautifulSoup

# 2. Instantiate Model

In [14]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [24]:
tokens = tokenizer.encode('This is amazing, I love it. GREAT!', return_tensors='pt')

In [25]:
#tokens[0]

In [26]:
#tokenizer.decode(tokens[0])

In [27]:
result = model(tokens)

In [28]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.4825, -2.9365, -1.5084,  1.3381,  4.5352]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [29]:
result.logits

tensor([[-2.4825, -2.9365, -1.5084,  1.3381,  4.5352]],
       grad_fn=<AddmmBackward0>)

In [30]:
torch.argmax(result.logits) #specifies the highest value

tensor(4)

In [31]:
int(torch.argmax(result.logits))+1

5

# 4. Collect Reviews

In [36]:
r= requests.get('http://www.yelp.com/biz/mejico-sydney-2')
soup= BeautifulSoup(r.text, 'html.parser')
regex=re.compile('.*comment.*')
results=soup.find_all('p',{'class':regex})
reviews=[result.text for result in results ]

In [37]:
results[0].text

'Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.'

In [39]:
reviews[2]

'Visiting from Texas and decided to give this restaurant a try. \xa0We were pleasantly surprised. \xa0While the margaritas are more like martinis, the food was excellent. \xa0More like a tapas, Mexican fusion. \xa0Great way to try different plates.The real treat was Chelsea, our waitress. \xa0Took the time to explain the menu and offer suggestions. \xa0Always smiling and very pleasant. \xa0Best service we have had in Sydney!'

# 5. Load reviews into DataFrame 

In [40]:
import pandas as pd
import numpy as np

In [42]:
df=pd.DataFrame(np.array(reviews),columns=['review'])

In [44]:
df.head()

,review
0,Hotter than I imagine myself to be and firing ...
1,The food and service here was really good. It...
2,Visiting from Texas and decided to give this r...
3,"Great atmosphere, attentive service, solid mar..."
4,Don't come here expecting legit Mexican food b...


In [45]:
df.iloc[0]

review    Hotter than I imagine myself to be and firing ...
Name: 0, dtype: object

In [46]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [47]:
sentiment_score(df['review'].iloc[0])

5

In [50]:
df['sentiment']=df['review'].apply(lambda x:sentiment_score(x[:512]))

In [51]:
df['review']

0    Hotter than I imagine myself to be and firing ...
1    The food and service here was really good.  It...
2    Visiting from Texas and decided to give this r...
3    Great atmosphere, attentive service, solid mar...
4    Don't come here expecting legit Mexican food b...
5    Out of all the restaurants that I tried in Syd...
6    We came here on a Thursday night @ 5pm and by ...
7    The food is fresh and tasty.  The scallop cevi...
8    Have been here twice and have absolutely loved...
9    I was pleasantly surprised at what a great job...
Name: review, dtype: object

In [52]:
df

,review,sentiment
0,Hotter than I imagine myself to be and firing ...,5
1,The food and service here was really good. It...,5
2,Visiting from Texas and decided to give this r...,5
3,"Great atmosphere, attentive service, solid mar...",3
4,Don't come here expecting legit Mexican food b...,3
5,Out of all the restaurants that I tried in Syd...,5
6,We came here on a Thursday night @ 5pm and by ...,4
7,The food is fresh and tasty. The scallop cevi...,4
8,Have been here twice and have absolutely loved...,5
9,I was pleasantly surprised at what a great job...,5
